In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer

In [2]:
from huggingface_hub import login
login()

In [3]:
if torch.cuda.is_available():
    model_id = "meta-llama/Llama-2-7b-chat-hf"
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
    model.cuda()
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
def chat_with_llama(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    input_ids = input_ids.to('cuda')
    output = model.generate(input_ids, max_length=512, num_beams=4, no_repeat_ngram_size=2)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [7]:
prompt = input("You: ")
response = chat_with_llama(prompt)
print("Llama:", response)

You:  Hello


RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 15.89 GiB total capacity; 14.24 GiB already allocated; 12.69 MiB free; 14.96 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [17]:
import torch
torch.cuda.empty_cache()

In [22]:
pip install llama-cpp-python

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.4/37.4 MB 7.1 MB/s eta 0:00:00m eta 0:00:010:00:01m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.4 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.61-cp39-cp39-manylinux_2_27_x86_64.whl size=3025057 sha256=4d49682efd105aea27d39f2f783f56f6942e8c3c7ebed611fb875c8e4dfd6116
  Stored in directory: /home/cv/.cache/pip/wheels/4a/00/cd/2163043edf9c1af59c44cd88207b049a3ceadff49d02b9e27f
Successfully built llama-cpp-python
Note: you may need to restart the kernel to use updated packages.


In [23]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate

In [26]:
template = """Question: {question}

Answer: Let's work this out in a step by step way to be sure we have the right answer."""

prompt = PromptTemplate.from_template(template)

In [27]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [40]:
n_gpu_layers = 1  # The number of layers to put on the GPU. The rest will be on the CPU. If you don't know how many layers there are, you can use -1 to move all to GPU.
n_batch = 1024  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="/home/cv/FYP_Chatbot/test02/open-source-models/llama2-7b/llama.cpp/llama-2-7b-chat.Q4_K_M.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
    n_ctx=2048,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /home/cv/FYP_Chatbot/test02/open-source-models/llama2-7b/llama.cpp/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv 

In [41]:
llm_chain = prompt | llm
question = "<s> [INST] <<SYS>>Generate questions and answers from given text<</SYS>>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n ### Instruction:\nYour task is to generate questions from the given text. Also answer these questions using the text given as input.\n### Input:\na mechanism of intermediate rotation which enables radiation, meaning a loss in their efficiency,\ntracking of the sun using an azimuth angle. which will be higher as the concentration factor\nis increased. The temperature is also affected\nby wind speed.\n26• Diffuse radiation: on days when radiation is After analysing the full economic viability, also in view of\nscarce due to clouds or fog, the diffuse radiation, the results obtained on prototype systems, the consortium\nwith an optical concentration closer to the unity concluded that the foreseen concept of a photovoltaic\nin the centre of the panel, becomes important system tracking technology with low concentration ratio\ncomparing it with the direct radiation, producing using standard modules does not lead to the expected\na strong decrease in the global optical reduction in electricity generation costs.\nconcentration.\n• String resistance: the cells present an internal\nresistance called ‘string resistance’ in most part\ndue to the metal net, contacts and the resistance\nof the semiconductor material. There is a thresh-\nold irradiance above which the losses in string\nINFORMATION\nresistance increase. As a result of this irradiance,\nthe efficiency of the cells will be maximal.\nIn particular the modules used in the prototype Project SOLAR-PLOTS\npresent a maximum efficiency for an irradiance acronym\nvalue of 600 W/m2. Thus, when working with Project Multiple ownership grid\nconcentration, the efficiency of the modules full title connected PV solar-plots\nwith optimised tracking and\nis reduced. In this case the gain loss is approxi-\nlow concentration refl ectors\nmately 13 % in the string with the geometric\nProposal/ 503172\nconcentration 2x and some 4.6 % in the string\ncontract no.\nwith the geometric concentration 1.5x.\nCoordinator Acciona Solar SA, Spain\nHowever, the temperature increase experienced\nContact: Miguel Arraras\nby the modules brings about one more problem.\nTotal EUR 4 824 274\nThe highest temperature guaranteed to maintain eligible cost\nthe integrity of the module is 85˚C. This would\nEU EUR 1 800 000\nbe achieved for the string with the geometric contribution\nconcentration 2x for an outside temperature of\nStart date June 2004\n30˚C; as such, the integrity of the modules will\nFinish date June 2006\nbe seriously at risk during the summer months.\nPartners Tenesol SA, France\nJoão Nuno Serra, Sociedade\nThe project found that the technical feasibility for concen-\nUnipessoal Lda, Portugal\ntration ratios of 2.5x or higher, with the current standard\nElektro Primorska Javno Podjetje Za\nphotovoltaic module technologies is questionable. The Distribucijo Elektricne Energije, D.D.,\nSlovenia\nmodule temperatures reached at this sun concentration\nCommissariat à l'énergie atomique,\nwill very probably lead to deterioration in the module prop-\nFrance\nerties. Even for lower concentration ratios, the situation\nCIEMAT, Centro de Investigaciones\nremains problematic. The photov oltaic module suppliers Energéticas, Medioambientales\ny Tecnológicas, Spain\nare not prepared to provide the usual guarantees (for\nCener, Centro Nacional de Energías\ninstance lifetime guarantees) for module performance, if\nRenovables – Fundación Cener,\nthe modules are used for con centration systems. Even with\nSpain\nmodules especially manufactured for concentration, guar-\nWebsite http://cordis.europa.eu/\nantees will not be provided.\n27DEMONSTRATION\nPROJECTS\nSolsilc Demonstrator\nValidation of a direct route for production of solar-\ngrade silicon feedstock for crystalline wafers and cells\nScenarios show that solar energy will be, in the long term,\nthe most important energy source, provided that the cost\nof photovoltaic modules is substantially decreased.\nL ow feedstock availability (priced from 35 EUR/kg\nto 100 EUR/kg) is currently jeopardising the\ndecreasing of module costs. Currently 17 % of\nmodule costs relate to feedstock. Even though cells will\nbe thinner and more efficient, demand for solar grade\n\n\n### Response:[/INST]"
llm_chain.invoke({"question": question})

 Please clarify which question you would like me to answer or provide further context on how to approach the task. 


llama_print_timings:        load time =  130534.64 ms
llama_print_timings:      sample time =      26.48 ms /    23 runs   (    1.15 ms per token,   868.45 tokens per second)
llama_print_timings: prompt eval time =  154002.48 ms /  1169 tokens (  131.74 ms per token,     7.59 tokens per second)
llama_print_timings:        eval time =  200504.98 ms /    22 runs   ( 9113.86 ms per token,     0.11 tokens per second)
llama_print_timings:       total time =  354777.81 ms /  1191 tokens


' Please clarify which question you would like me to answer or provide further context on how to approach the task. '

In [31]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on -D CMAKE_CUDA_COMPILER=$(which nvcc)" FORCE_CMAKE=1 pip install --upgrade --force-reinstall llama-cpp-python --no-cache-dir

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.4/37.4 MB 10.2 MB/s eta 0:00:00m eta 0:00:0136m0:00:01m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 9.6 MB/s eta 0:00:000:00:010:00:01:01m
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.61-cp311-cp311-manylinux_2_27_x86_64.whl size=39182885 sha256=0b8f457a3bbc7d1cd2d89d8b36aeb199bdef5637ea45e059b694dadd8e11df84
  Stored in directory: /tmp/pip-ephem-wheel-cache-xk2afk2e/wheels/94/0f/18/819e646fe2f6c66d83f28574794b2be811f4162f99da513515
Successfully built llama-cpp-python
  Attempting uninstall:

In [5]:
import subprocess

# Run the nvidia-smi command and capture the output
output = subprocess.check_output(['nvidia-smi'])

# Print the output
print(output.decode('utf-8'))

Fri Apr 12 14:40:23 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.182.03   Driver Version: 470.182.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P5000        Off  | 00000000:01:00.0  On |                  Off |
| 27%   43C    P8    11W / 180W |    413MiB / 16275MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [29]:
import torch
torch.cuda.is_available()

False

In [30]:
pip show torch

Name: torch
Version: 1.12.1
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /home/cv/FYP_Chatbot/test02/src/myenv-test002-03/lib/python3.9/site-packages
Requires: typing-extensions
Required-by: accelerate, bitsandbytes, peft, trl
Note: you may need to restart the kernel to use updated packages.
